# 🧪 Validación Matemática - Función Lambda AquaSense (NUEVA VERSIÓN)

Este notebook valida que los cálculos estadísticos de la **nueva función Lambda** sean correctos.

**Nueva funcionalidad:**
- Lee TODOS los archivos CSV del bucket
- Detecta y sobrescribe fechas duplicadas (última aparición gana)
- **Ajuste de mes: si el día es <= 3, se asigna al mes anterior**
- Recalcula métricas con todos los datos fusionados

**Métricas validadas:**
1. `max_temp` - Temperatura máxima del mes
2. `max_sd` - Desviación estándar máxima
3. `mean_temp` - Temperatura media del mes
4. `max_diff_temp` - Diferencia con mes anterior

## 📦 Importaciones y Configuración

In [1]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from decimal import Decimal
from pathlib import Path
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Configuración
DATA_DIR = Path('../Data')
DEVIATION_THRESHOLD = Decimal("0.5")

print("✅ Librerías importadas correctamente")
print(f"📁 Directorio de datos: {DATA_DIR}")
print(f"🌡️  Umbral de desviación: {DEVIATION_THRESHOLD}°C")

✅ Librerías importadas correctamente
📁 Directorio de datos: ../Data
🌡️  Umbral de desviación: 0.5°C


## 🔧 Funciones Lambda (Copiadas)

In [2]:
def round_decimal(value):
    """Redondea un valor Decimal a 2 decimales (EXACTO DE LA LAMBDA)."""
    return value.quantize(Decimal("0.0001"))

def adjust_month_for_date(fecha_dt):
    """
    Ajusta el mes de una fecha según el día.
    Si el día es <= 3, se asigna al mes anterior.
    
    Args:
        fecha_dt: datetime object
    
    Returns:
        str: Mes en formato 'YYYY-MM' ajustado
    """
    if fecha_dt.day <= 3:
        # Mover al mes anterior
        adjusted_date = fecha_dt.replace(day=1) - timedelta(days=1)
        return adjusted_date.strftime('%Y-%m')
    else:
        return fecha_dt.strftime('%Y-%m')

print("✅ Funciones Lambda definidas")

✅ Funciones Lambda definidas


## 📊 Función de Procesamiento (Lógica Lambda)

In [3]:
def process_csv_lambda_logic(filepath):
    """
    Procesa un CSV usando la NUEVA lógica Lambda.
    Simula: leer archivo, detectar duplicados por fecha (última gana).
    Retorna datos por fecha individual para fusionar después.
    """
    daily_data = {}  # {'2023-01-15': {'temp': 20, 'sd': 0.5, 'adjusted_month': '2023-01'}}
    
    with open(filepath, encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',')
        
        for row in reader:
            # Parseo de fecha (EXACTO DE LA LAMBDA)
            try:
                fecha = datetime.strptime(row['Fecha'], '%Y/%m/%d')
            except ValueError:
                try:
                    fecha = datetime.strptime(row['Fecha'], '%Y-%m-%d')
                except ValueError:
                    print(f"Warning: Invalid date format: {row['Fecha']}")
                    continue
            
            fecha_str = fecha.strftime('%Y-%m-%d')
            temp_media = Decimal(row['Medias'])
            desviacion = Decimal(row['Desviaciones'])
            
            # Ajustar mes según el día (si día <= 3, va al mes anterior)
            adjusted_month = adjust_month_for_date(fecha)
            
            # SOBRESCRIBE si la fecha ya existe (NUEVA LÓGICA)
            daily_data[fecha_str] = {
                'temp': temp_media,
                'sd': desviacion,
                'source': filepath.name,
                'adjusted_month': adjusted_month
            }
    
    return daily_data

def process_multiple_files_lambda_logic(filepaths):
    """
    Simula el procesamiento de MÚLTIPLES archivos como hace la nueva Lambda.
    Fusiona datos de todos los archivos, última aparición gana.
    """
    merged_daily_data = {}
    alerts = []
    month_adjustments = 0
    
    # Procesar todos los archivos en orden alfabético
    for filepath in sorted(filepaths):
        file_data = process_csv_lambda_logic(filepath)
        
        # Fusionar: última aparición sobrescribe
        for fecha_str, data in file_data.items():
            if fecha_str in merged_daily_data:
                print(f"  → Duplicate date {fecha_str}: overwriting {merged_daily_data[fecha_str]['source']} with {filepath.name}")
            
            # Verificar si se ajustó el mes
            fecha_dt = datetime.strptime(fecha_str, '%Y-%m-%d')
            original_month = fecha_dt.strftime('%Y-%m')
            if data['adjusted_month'] != original_month:
                month_adjustments += 1
                print(f"  → Date {fecha_str} (day {fecha_dt.day}) assigned to month {data['adjusted_month']} (was {original_month})")
            
            merged_daily_data[fecha_str] = data
            
            # Detectar alertas
            if data['sd'] > DEVIATION_THRESHOLD:
                fecha_dt = datetime.strptime(fecha_str, '%Y-%m-%d')
                alerts.append({
                    'fecha': fecha_dt,
                    'desviacion': data['sd'],
                    'temp_media': data['temp']
                })
    
    # Agrupar por mes AJUSTADO y calcular métricas
    monthly_data = defaultdict(dict)
    for fecha_str, data in merged_daily_data.items():
        # Usar el mes ajustado en lugar del mes natural
        mes = data['adjusted_month']
        monthly_data[mes][fecha_str] = data
    
    # Calcular métricas agregadas por mes (como hace Lambda)
    aggregated_data = {}
    raw_data = defaultdict(list)
    
    for mes, dates_dict in monthly_data.items():
        all_temps = [d['temp'] for d in dates_dict.values()]
        all_sds = [d['sd'] for d in dates_dict.values()]
        
        # Guardar raw_data para validación
        for fecha_str, d in dates_dict.items():
            fecha_dt = datetime.strptime(fecha_str, '%Y-%m-%d')
            raw_data[mes].append({
                'fecha': fecha_dt,
                'temp': d['temp'],
                'sd': d['sd']
            })
        
        aggregated_data[mes] = {
            'max_temp': max(all_temps),
            'max_sd': max(all_sds),
            'mean_temp_sum': sum(all_temps),
            'mean_temp_count': len(all_temps)
        }
    
    print(f"\n📊 Ajustes de mes (día <= 3): {month_adjustments}")
    
    return aggregated_data, raw_data, alerts, merged_daily_data

print("✅ Funciones de procesamiento definidas (NUEVA LÓGICA con ajuste de mes)")

✅ Funciones de procesamiento definidas (NUEVA LÓGICA con ajuste de mes)


## 🧮 Función de Validación Matemática

In [4]:
def validate_mathematics(data, raw_data, verbose=True):
    """
    Valida que todos los cálculos matemáticos sean correctos.
    Compara resultados de la Lambda vs cálculos manuales.
    """
    results = []
    meses_ordenados = sorted(data.keys())
    
    for i, mes in enumerate(meses_ordenados):
        metrics = data[mes]
        raw = raw_data[mes]
        validaciones = {'mes': mes, 'errores': []}
        
        if verbose:
            print(f"\n{'='*60}")
            print(f"📅 Mes: {mes} ({len(raw)} registros)")
            print(f"{'='*60}")
        
        # ========== VALIDACIÓN 1: max_temp ==========
        max_temp_lambda = metrics['max_temp']
        max_temp_real = max(r['temp'] for r in raw)
        
        if max_temp_lambda == max_temp_real:
            if verbose:
                print(f"✅ max_temp: {float(max_temp_lambda):.2f}°C")
            validaciones['max_temp'] = 'OK'
        else:
            error = f"max_temp: Lambda={float(max_temp_lambda):.2f}, Real={float(max_temp_real):.2f}"
            if verbose:
                print(f"❌ {error}")
            validaciones['errores'].append(error)
            validaciones['max_temp'] = 'ERROR'
        
        # ========== VALIDACIÓN 2: max_sd ==========
        max_sd_lambda = metrics['max_sd']
        max_sd_real = max(r['sd'] for r in raw)
        
        if max_sd_lambda == max_sd_real:
            if verbose:
                print(f"✅ max_sd: {float(max_sd_lambda):.4f}°C")
            validaciones['max_sd'] = 'OK'
        else:
            error = f"max_sd: Lambda={float(max_sd_lambda):.4f}, Real={float(max_sd_real):.4f}"
            if verbose:
                print(f"❌ {error}")
            validaciones['errores'].append(error)
            validaciones['max_sd'] = 'ERROR'
        
        # ========== VALIDACIÓN 3: mean_temp ==========
        mean_temp_lambda = metrics['mean_temp_sum'] / metrics['mean_temp_count']
        
        suma_temps = sum(r['temp'] for r in raw)
        count_temps = len(raw)
        mean_temp_real = suma_temps / count_temps
        
        if mean_temp_lambda == mean_temp_real:
            if verbose:
                print(f"✅ mean_temp: {float(mean_temp_lambda):.2f}°C (n={count_temps})")
                print(f"   └─ Σ={float(suma_temps):.2f}, n={count_temps}")
            validaciones['mean_temp'] = 'OK'
        else:
            error = f"mean_temp: Lambda={float(mean_temp_lambda):.2f}, Real={float(mean_temp_real):.2f}"
            if verbose:
                print(f"❌ {error}")
            validaciones['errores'].append(error)
            validaciones['mean_temp'] = 'ERROR'
        
        # ========== VALIDACIÓN 4: max_diff_temp ==========
        if i > 0:
            mes_anterior = meses_ordenados[i-1]
            
            # Lógica Lambda: buscar mes anterior en archivo
            if mes_anterior in data:
                prev_max_file = data[mes_anterior]['max_temp']
            else:
                prev_max_file = Decimal(0)
            
            # En Lambda también busca en DB, aquí solo archivo
            previous_max_final = prev_max_file
            
            max_diff_lambda = metrics['max_temp'] - previous_max_final
            max_diff_real = max_temp_lambda - data[mes_anterior]['max_temp']
            
            if max_diff_lambda == max_diff_real:
                if verbose:
                    print(f"✅ max_diff_temp: {float(max_diff_lambda):+.2f}°C")
                    print(f"   └─ {float(max_temp_lambda):.2f} - {float(data[mes_anterior]['max_temp']):.2f}")
                validaciones['max_diff_temp'] = 'OK'
            else:
                error = f"max_diff_temp: Lambda={float(max_diff_lambda):+.2f}, Real={float(max_diff_real):+.2f}"
                if verbose:
                    print(f"❌ {error}")
                validaciones['errores'].append(error)
                validaciones['max_diff_temp'] = 'ERROR'
        else:
            if verbose:
                print(f"⚠️  max_diff_temp: N/A (primer mes)")
            validaciones['max_diff_temp'] = 'N/A'
        
        results.append(validaciones)
    
    return results

print("✅ Función de validación definida")

✅ Función de validación definida


## 🔍 Seleccionar Archivo a Probar

In [5]:
# Lista de archivos disponibles
csv_files = sorted(DATA_DIR.glob("*.csv"))

print(f"📁 Archivos CSV disponibles ({len(csv_files)}):")
for i, f in enumerate(csv_files, 1):
    print(f"   {i}. {f.name}")

# Seleccionar archivos a procesar (simular múltiples archivos en bucket)
# Cambiar el rango según necesites: csv_files[0:3] procesa los primeros 3
archivos_seleccionados = csv_files[0:5]  # Por defecto primeros 5

# AÑADIR MANUALMENTE temperatura_pruebas.csv si existe
archivo_pruebas = DATA_DIR / "temperatura_pruebas.csv"
if archivo_pruebas.exists() and archivo_pruebas not in archivos_seleccionados:
    archivos_seleccionados = list(archivos_seleccionados) + [archivo_pruebas]
    print(f"\n📌 Añadido manualmente: temperatura_pruebas.csv")

print(f"\n🎯 Archivos seleccionados para procesamiento: {len(archivos_seleccionados)}")
for f in archivos_seleccionados:
    print(f"   - {f.name}")

📁 Archivos CSV disponibles (22):
   1. Temperatura.csv
   2. temperatura_1.csv
   3. temperatura_10.csv
   4. temperatura_11.csv
   5. temperatura_12.csv
   6. temperatura_13.csv
   7. temperatura_14.csv
   8. temperatura_15.csv
   9. temperatura_16.csv
   10. temperatura_17.csv
   11. temperatura_18.csv
   12. temperatura_19.csv
   13. temperatura_2.csv
   14. temperatura_20.csv
   15. temperatura_3.csv
   16. temperatura_4.csv
   17. temperatura_5.csv
   18. temperatura_6.csv
   19. temperatura_7.csv
   20. temperatura_8.csv
   21. temperatura_9.csv
   22. temperatura_pruebas.csv

📌 Añadido manualmente: temperatura_pruebas.csv

🎯 Archivos seleccionados para procesamiento: 6
   - Temperatura.csv
   - temperatura_1.csv
   - temperatura_10.csv
   - temperatura_11.csv
   - temperatura_12.csv
   - temperatura_pruebas.csv


## 🚀 Ejecutar Validación Completa

In [6]:
# Procesar MÚLTIPLES archivos con lógica Lambda (NUEVA VERSIÓN)
print(f"\n{'='*70}")
print(f"🔄 PROCESANDO {len(archivos_seleccionados)} ARCHIVOS (SIMULA BUCKET S3)")
print(f"{'='*70}")

data, raw_data, alerts, merged_daily_data = process_multiple_files_lambda_logic(archivos_seleccionados)

print(f"\n📊 Resumen de procesamiento:")
print(f"   - Archivos procesados: {len(archivos_seleccionados)}")
print(f"   - Fechas únicas (después de fusionar): {len(merged_daily_data)}")
print(f"   - Meses encontrados: {len(data)}")
print(f"   - Total de registros por mes: {sum(len(raw_data[m]) for m in raw_data)}")
print(f"   - Alertas detectadas: {len(alerts)}")

# Ejecutar validación
resultados = validate_mathematics(data, raw_data, verbose=True)


🔄 PROCESANDO 6 ARCHIVOS (SIMULA BUCKET S3)
  → Date 2017-06-02 (day 2) assigned to month 2017-05 (was 2017-06)
  → Date 2017-11-03 (day 3) assigned to month 2017-10 (was 2017-11)
  → Date 2018-01-02 (day 2) assigned to month 2017-12 (was 2018-01)
  → Date 2018-05-02 (day 2) assigned to month 2018-04 (was 2018-05)
  → Date 2018-08-03 (day 3) assigned to month 2018-07 (was 2018-08)
  → Date 2018-10-03 (day 3) assigned to month 2018-09 (was 2018-10)
  → Date 2019-01-03 (day 3) assigned to month 2018-12 (was 2019-01)
  → Date 2019-10-03 (day 3) assigned to month 2019-09 (was 2019-10)
  → Date 2019-12-02 (day 2) assigned to month 2019-11 (was 2019-12)
  → Date 2020-01-03 (day 3) assigned to month 2019-12 (was 2020-01)
  → Date 2020-04-01 (day 1) assigned to month 2020-03 (was 2020-04)
  → Date 2020-06-01 (day 1) assigned to month 2020-05 (was 2020-06)
  → Date 2020-06-02 (day 2) assigned to month 2020-05 (was 2020-06)
  → Date 2020-07-02 (day 2) assigned to month 2020-06 (was 2020-07)
  → 

## 📈 Resumen de Validaciones

In [7]:
# Crear DataFrame de resultados
df_results = pd.DataFrame([
    {
        'Mes': r['mes'],
        'max_temp': r.get('max_temp', 'N/A'),
        'max_sd': r.get('max_sd', 'N/A'),
        'mean_temp': r.get('mean_temp', 'N/A'),
        'max_diff_temp': r.get('max_diff_temp', 'N/A'),
        'Errores': len(r['errores'])
    }
    for r in resultados
])

print(f"\n{'='*70}")
print("📊 TABLA DE RESULTADOS")
print(f"{'='*70}\n")
print(df_results.to_string(index=False))

# Resumen final
total_validaciones = len(resultados)
validaciones_ok = sum(1 for r in resultados if len(r['errores']) == 0)
validaciones_error = total_validaciones - validaciones_ok

print(f"\n{'='*70}")
print("🎯 RESUMEN FINAL")
print(f"{'='*70}")
print(f"   Total de meses validados: {total_validaciones}")
print(f"   ✅ Meses sin errores: {validaciones_ok}")
print(f"   ❌ Meses con errores: {validaciones_error}")

if validaciones_error == 0:
    print(f"\n   🎉 ¡TODAS LAS VALIDACIONES PASARON!")
else:
    print(f"\n   ⚠️  HAY ERRORES - Revisar cálculos")
    print(f"\n   Detalles de errores:")
    for r in resultados:
        if r['errores']:
            print(f"\n   📅 {r['mes']}:")
            for e in r['errores']:
                print(f"      - {e}")


📊 TABLA DE RESULTADOS

    Mes max_temp max_sd mean_temp max_diff_temp  Errores
2017-03       OK     OK        OK           N/A        0
2017-04       OK     OK        OK            OK        0
2017-05       OK     OK        OK            OK        0
2017-06       OK     OK        OK            OK        0
2017-07       OK     OK        OK            OK        0
2017-08       OK     OK        OK            OK        0
2017-09       OK     OK        OK            OK        0
2017-10       OK     OK        OK            OK        0
2017-11       OK     OK        OK            OK        0
2017-12       OK     OK        OK            OK        0
2018-01       OK     OK        OK            OK        0
2018-02       OK     OK        OK            OK        0
2018-03       OK     OK        OK            OK        0
2018-04       OK     OK        OK            OK        0
2018-05       OK     OK        OK            OK        0
2018-06       OK     OK        OK            OK        0
2018-07

## 🔔 Análisis de Alertas

In [8]:
if alerts:
    df_alerts = pd.DataFrame([
        {
            'Fecha': a['fecha'].strftime('%Y-%m-%d'),
            'Desviación (°C)': float(a['desviacion']),
            'Temp Media (°C)': float(a['temp_media']),
            'Exceso vs Umbral (°C)': float(a['desviacion'] - DEVIATION_THRESHOLD)
        }
        for a in alerts
    ])
    
    print(f"\n{'='*70}")
    print(f"🔔 ALERTAS DETECTADAS ({len(alerts)})")
    print(f"{'='*70}\n")
    print(df_alerts.to_string(index=False))
    
    print(f"\n📊 Estadísticas de alertas:")
    print(f"   - Desviación máxima: {df_alerts['Desviación (°C)'].max():.4f}°C")
    print(f"   - Desviación promedio: {df_alerts['Desviación (°C)'].mean():.4f}°C")
    print(f"   - Umbral configurado: {float(DEVIATION_THRESHOLD):.2f}°C")
else:
    print(f"\n✅ No se detectaron alertas (todas las desviaciones < {float(DEVIATION_THRESHOLD):.2f}°C)")


🔔 ALERTAS DETECTADAS (43)

     Fecha  Desviación (°C)  Temp Media (°C)  Exceso vs Umbral (°C)
2017-04-05         0.625379        18.244801               0.125379
2017-08-30         0.527440        27.431929               0.027440
2017-09-06         0.520555        27.129522               0.020555
2017-10-11         0.506468        24.193134               0.006468
2017-12-05         0.908412        11.415262               0.408412
2018-06-20         0.655443        26.350178               0.155443
2018-10-03         0.570327        24.824747               0.070327
2018-12-26         0.725702        13.245550               0.225702
2019-01-17         0.568646        11.405227               0.068646
2019-08-29         0.507686        27.690094               0.007686
2019-09-17         0.718972        24.283982               0.218972
2019-09-19         0.696657        24.512966               0.196657
2019-09-27         0.649990        25.884669               0.149990
2019-10-15         0

## 📊 Visualización de Datos Procesados

In [9]:
# Crear DataFrame con métricas mensuales
df_metrics = pd.DataFrame([
    {
        'Mes': mes,
        'Max Temp (°C)': float(metrics['max_temp']),
        'Max SD (°C)': float(metrics['max_sd']),
        'Mean Temp (°C)': float(metrics['mean_temp_sum'] / metrics['mean_temp_count']),
        'Registros': metrics['mean_temp_count']
    }
    for mes, metrics in sorted(data.items())
])

print(f"\n{'='*70}")
print("📊 MÉTRICAS MENSUALES CALCULADAS")
print(f"{'='*70}\n")
print(df_metrics.to_string(index=False))

print(f"\n📈 Estadísticas globales:")
print(f"   - Temperatura máxima absoluta: {df_metrics['Max Temp (°C)'].max():.2f}°C")
print(f"   - Temperatura media global: {df_metrics['Mean Temp (°C)'].mean():.2f}°C")
print(f"   - Desviación máxima: {df_metrics['Max SD (°C)'].max():.4f}°C")


📊 MÉTRICAS MENSUALES CALCULADAS

    Mes  Max Temp (°C)  Max SD (°C)  Mean Temp (°C)  Registros
2017-03      17.329895     0.403720       17.056984          2
2017-04      19.465319     0.625379       18.872127          3
2017-05      24.581282     0.390436       22.561428          4
2017-06      28.228497     0.413879       26.761327          4
2017-07      29.515522     0.323097       28.274396          4
2017-08      27.431929     0.527440       27.431929          1
2017-09      27.129522     0.520555       25.909508          3
2017-10      24.875645     0.506468       23.118340          5
2017-11      18.211737     0.498608       16.238055          4
2017-12      12.216244     0.908412       11.730347          5
2018-01      12.573280     0.423534       12.122393          4
2018-02      12.962960     0.421105       12.224599          3
2018-03      15.187907     0.241254       13.966333          3
2018-04      18.557665     0.449467       17.770530          4
2018-05      23.78431

## 🧪 Test de Duplicados (Validación Crítica)

In [10]:
# Test crítico: Verificar que se detectan y sobrescriben duplicados correctamente
print(f"\n{'='*70}")
print("🧪 VALIDACIÓN DE LÓGICA DE DUPLICADOS Y AJUSTE DE MES")
print(f"{'='*70}\n")

# Crear archivos de prueba simulados con fechas duplicadas
import tempfile
import os

# Crear directorio temporal
temp_dir = Path(tempfile.mkdtemp())

# Archivo 1: Datos iniciales (incluye fechas con día <= 3)
archivo1_path = temp_dir / "test_1.csv"
with open(archivo1_path, 'w', encoding='utf-8') as f:
    f.write("Fecha,Medias,Desviaciones\n")
    f.write("2023-01-05,18.0,0.3\n")
    f.write("2023-01-15,20.0,0.4\n")
    f.write("2023-02-02,17.0,0.25\n")  # Día 2, debe ir a 2023-01

# Archivo 2: Con fecha duplicada (debe sobrescribir)
archivo2_path = temp_dir / "test_2.csv"
with open(archivo2_path, 'w', encoding='utf-8') as f:
    f.write("Fecha,Medias,Desviaciones\n")
    f.write("2023-01-15,22.0,0.5\n")  # Sobrescribe la del archivo 1
    f.write("2023-01-25,19.0,0.35\n")
    f.write("2023-03-01,16.0,0.28\n")  # Día 1, debe ir a 2023-02

test_files = [archivo1_path, archivo2_path]

print("📝 Archivos de prueba creados:")
print(f"   Archivo 1: 2023-01-05=18°C, 2023-01-15=20°C, 2023-02-02=17°C")
print(f"   Archivo 2: 2023-01-15=22°C, 2023-01-25=19°C, 2023-03-01=16°C")
print(f"\n   ✅ ESPERADO: 2023-01-15 debe tener 22°C (del archivo 2)")
print(f"   ✅ ESPERADO: 2023-02-02 asignado a mes 2023-01 (día <= 3)")
print(f"   ✅ ESPERADO: 2023-03-01 asignado a mes 2023-02 (día <= 3)")

# Procesar
data_test, raw_test, alerts_test, merged_test = process_multiple_files_lambda_logic(test_files)

# Verificar
print(f"\n📊 Resultados:")
print(f"   - Fechas únicas procesadas: {len(merged_test)}")
print(f"   - Fecha 2023-01-15 tiene: {float(merged_test['2023-01-15']['temp']):.2f}°C")
print(f"   - Fecha 2023-02-02 asignada al mes: {merged_test['2023-02-02']['adjusted_month']}")
print(f"   - Fecha 2023-03-01 asignada al mes: {merged_test['2023-03-01']['adjusted_month']}")

# Validar meses
print(f"\n📅 Métricas por mes ajustado:")
for mes in sorted(data_test.keys()):
    mean_calc = float(data_test[mes]['mean_temp_sum'] / data_test[mes]['mean_temp_count'])
    print(f"   {mes}: {data_test[mes]['mean_temp_count']} registros, mean={mean_calc:.2f}°C")

# Validación final
validacion_exitosa = True
if len(merged_test) != 5:
    print(f"\n❌ ERROR: Esperadas 5 fechas únicas, encontradas {len(merged_test)}")
    validacion_exitosa = False

if float(merged_test['2023-01-15']['temp']) != 22.0:
    print(f"\n❌ ERROR: Duplicado no sobrescrito correctamente")
    validacion_exitosa = False

if merged_test['2023-02-02']['adjusted_month'] != '2023-01':
    print(f"\n❌ ERROR: 2023-02-02 no asignado correctamente al mes anterior")
    validacion_exitosa = False

if merged_test['2023-03-01']['adjusted_month'] != '2023-02':
    print(f"\n❌ ERROR: 2023-03-01 no asignado correctamente al mes anterior")
    validacion_exitosa = False

if validacion_exitosa:
    print(f"\n✅ ¡VALIDACIÓN EXITOSA! Los duplicados y ajustes de mes funcionan correctamente")

# Limpiar archivos temporales
import shutil
shutil.rmtree(temp_dir)
print(f"\n🗑️  Archivos de prueba eliminados")


🧪 VALIDACIÓN DE LÓGICA DE DUPLICADOS Y AJUSTE DE MES

📝 Archivos de prueba creados:
   Archivo 1: 2023-01-05=18°C, 2023-01-15=20°C, 2023-02-02=17°C
   Archivo 2: 2023-01-15=22°C, 2023-01-25=19°C, 2023-03-01=16°C

   ✅ ESPERADO: 2023-01-15 debe tener 22°C (del archivo 2)
   ✅ ESPERADO: 2023-02-02 asignado a mes 2023-01 (día <= 3)
   ✅ ESPERADO: 2023-03-01 asignado a mes 2023-02 (día <= 3)
  → Date 2023-02-02 (day 2) assigned to month 2023-01 (was 2023-02)
  → Duplicate date 2023-01-15: overwriting test_1.csv with test_2.csv
  → Date 2023-03-01 (day 1) assigned to month 2023-02 (was 2023-03)

📊 Ajustes de mes (día <= 3): 2

📊 Resultados:
   - Fechas únicas procesadas: 5
   - Fecha 2023-01-15 tiene: 22.00°C
   - Fecha 2023-02-02 asignada al mes: 2023-01
   - Fecha 2023-03-01 asignada al mes: 2023-02

📅 Métricas por mes ajustado:
   2023-01: 4 registros, mean=19.00°C
   2023-02: 1 registros, mean=16.00°C

✅ ¡VALIDACIÓN EXITOSA! Los duplicados y ajustes de mes funcionan correctamente

🗑️  

## 🔄 Probar con Otro Conjunto de Archivos (Opcional)

In [11]:
# Para probar con diferente conjunto de archivos:
archivos_test = csv_files[5:15]  # Cambiar rango según necesites

if len(archivos_test) > 0:
    print(f"🔄 Probando con {len(archivos_test)} archivos:")
    for f in archivos_test:
        print(f"   - {f.name}")
    print()
    
    data2, raw_data2, alerts2, merged2 = process_multiple_files_lambda_logic(archivos_test)
    resultados2 = validate_mathematics(data2, raw_data2, verbose=False)
    
    errores = sum(1 for r in resultados2 if len(r['errores']) > 0)
    print(f"\n{'✅' if errores == 0 else '❌'} Resultado: {len(resultados2)} meses, {errores} con errores")
    print(f"   Fechas únicas procesadas: {len(merged2)}")
else:
    print("⚠️  No hay suficientes archivos para probar este rango")

🔄 Probando con 10 archivos:
   - temperatura_13.csv
   - temperatura_14.csv
   - temperatura_15.csv
   - temperatura_16.csv
   - temperatura_17.csv
   - temperatura_18.csv
   - temperatura_19.csv
   - temperatura_2.csv
   - temperatura_20.csv
   - temperatura_3.csv

  → Date 2019-10-03 (day 3) assigned to month 2019-09 (was 2019-10)
  → Date 2019-12-02 (day 2) assigned to month 2019-11 (was 2019-12)
  → Date 2024-01-03 (day 3) assigned to month 2023-12 (was 2024-01)
  → Date 2024-02-01 (day 1) assigned to month 2024-01 (was 2024-02)
  → Date 2024-04-02 (day 2) assigned to month 2024-03 (was 2024-04)
  → Date 2022-02-02 (day 2) assigned to month 2022-01 (was 2022-02)
  → Date 2022-03-02 (day 2) assigned to month 2022-02 (was 2022-03)
  → Date 2020-01-03 (day 3) assigned to month 2019-12 (was 2020-01)
  → Date 2020-04-01 (day 1) assigned to month 2020-03 (was 2020-04)
  → Date 2019-01-03 (day 3) assigned to month 2018-12 (was 2019-01)
  → Date 2018-01-02 (day 2) assigned to month 2017-12